In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame

from sklearn.model_selection import train_test_split

import tensorflow as tf

In [2]:
DATA_DIR = './ml-100k'
LOG_DIR = './logs'
data = pd.read_csv('{}/u.data'.format(DATA_DIR), sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
data.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
train, val = train_test_split(data, test_size=0.2)
train.shape, val.shape

((80000, 4), (20000, 4))

In [4]:
USER_NUM = 943
ITEM_NUM = 1682
train_matrix = np.zeros((USER_NUM, ITEM_NUM))
val_matrix = np.zeros((USER_NUM, ITEM_NUM))
train_matrix.shape

(943, 1682)

In [5]:
for _, row in train.iterrows():
    user_id, item_id, rating = row.user_id, row.item_id, row.rating
    train_matrix[user_id-1, item_id-1] = rating
    
for _, row in val.iterrows():
    user_id, item_id, rating = row.user_id, row.item_id, row.rating
    val_matrix[user_id-1, item_id-1] = rating

In [6]:
sum_matrix = np.sum(train_matrix, axis=0)
count_matrix = np.count_nonzero(train_matrix, axis=0)

# Maybe you can try reduce all other items by 3(batch normalization), leave blank as 0 instead of 3.
sum_matrix[sum_matrix==0] = 3
count_matrix[count_matrix==0] = 1

item_avg = sum_matrix / count_matrix

matrix_mean = np.ndarray((USER_NUM, ITEM_NUM))
for i in range(USER_NUM):
    matrix_mean[i] = item_avg

In [7]:
FEATURE_LEN = 64

tf_train_matrix = tf.constant(train_matrix, dtype=tf.float32)
tf_train_matrix_bool = tf.cast(tf_train_matrix, dtype=tf.bool)
tf_val_matrix = tf.constant(val_matrix, dtype=tf.float32)
tf_val_matrix_bool = tf.cast(tf_val_matrix, dtype=tf.bool)

tf_user_features = tf.Variable(tf.truncated_normal(shape=(USER_NUM, FEATURE_LEN)), dtype=tf.float32)
tf_item_features = tf.Variable(tf.truncated_normal(shape=(ITEM_NUM, FEATURE_LEN)), dtype=tf.float32)

tf_matrix_mean = tf.constant(matrix_mean, dtype=tf.float32)
tf_matrix_zero_float = tf.constant(np.zeros((USER_NUM, ITEM_NUM)), dtype=tf.float32)
tf_matrix_zero_int = tf.constant(np.zeros((USER_NUM, ITEM_NUM)), dtype=tf.int32)

In [10]:
# Loss

pred_matrix = tf.tensordot(tf_user_features, tf_item_features, axes=[[1], [1]])
# pred_matrix_int = tf.cast(pred_matrix, tf.int32)
pred_matrix_int = tf.cast(tf.round(pred_matrix), tf.int32)

train_diff = pred_matrix - tf_train_matrix
train_squared_diff_filtered = tf.where(tf_train_matrix_bool, tf.square(train_diff), tf_matrix_mean)

val_diff = pred_matrix - tf_val_matrix
val_squared_diff_filtered = tf.where(tf_val_matrix_bool, tf.square(val_diff), tf_matrix_mean)

train_diff_loss = 0.1 * tf.sqrt(tf.reduce_sum(train_squared_diff_filtered))
train_user_loss = 0.1 * tf.sqrt(tf.reduce_sum(tf.square(tf_user_features)))
train_item_loss = 0.1 * tf.sqrt(tf.reduce_sum(tf.square(tf_item_features)))
train_loss = train_diff_loss + train_user_loss + train_item_loss

val_diff_loss = 0.1 * tf.sqrt(tf.reduce_sum(val_squared_diff_filtered))
val_user_loss = 0.1 * tf.sqrt(tf.reduce_sum(tf.square(tf_user_features)))
val_item_loss = 0.1 * tf.sqrt(tf.reduce_sum(tf.square(tf_item_features)))
val_loss = val_diff_loss + val_user_loss + val_item_loss


tf.summary.scalar('train_loss_diff', train_diff_loss)
tf.summary.scalar('train_loss_user', train_user_loss)
tf.summary.scalar('train_loss_item', train_item_loss)
tf.summary.scalar('train_loss_total', train_loss)

tf.summary.scalar('val_loss_diff', val_diff_loss)
tf.summary.scalar('val_loss_user', val_user_loss)
tf.summary.scalar('val_loss_item', val_item_loss)
tf.summary.scalar('val_loss_total', val_loss)

<tf.Tensor 'val_loss_total_1:0' shape=() dtype=string>

In [11]:
# Train & Val Accuracy
# Way1 Avg Dist
train_mean_abs_dist = tf.reduce_sum(tf.where(tf_train_matrix_bool, tf.abs(train_diff), tf_matrix_zero_float)) / 80000
val_mean_abs_dist = tf.reduce_sum(tf.where(tf_val_matrix_bool, tf.abs(val_diff), tf_matrix_zero_float)) / 20000

tf.summary.scalar('train_avg_dist', train_mean_abs_dist)
tf.summary.scalar('val_avg_dist', val_mean_abs_dist)

# Way2 Accuracy
train_equal_matrix = tf.cast(tf.equal(pred_matrix_int, tf.cast(tf_train_matrix, tf.int32)), tf.int32)
val_equal_matrix = tf.cast(tf.equal(pred_matrix_int, tf.cast(tf_val_matrix, tf.int32)), tf.int32)

train_equal_num = tf.reduce_sum(tf.where(tf_train_matrix_bool, train_equal_matrix, tf_matrix_zero_int))
val_equal_num = tf.reduce_sum(tf.where(tf_val_matrix_bool, val_equal_matrix, tf_matrix_zero_int))

tf.summary.scalar('train_equal_num', train_equal_num)
tf.summary.scalar('train_equal_ratio', train_equal_num / 80000)
tf.summary.scalar('val_equal_num', val_equal_num)
tf.summary.scalar('val_equal_ratio', val_equal_num / 20000)

<tf.Tensor 'val_equal_ratio:0' shape=() dtype=string>

In [12]:
# train_op = tf.train.AdadeltaOptimizer(learning_rate=1).minimize(train_loss)
train_op = tf.train.GradientDescentOptimizer(learning_rate=1).minimize(train_loss)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

RUN = RUN + 1 if 'RUN' in locals() else 1
print('RUN {}'.format(RUN))
summary_op = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter("{}/RUN{}/".format(LOG_DIR, RUN), sess.graph)

RUN 1


In [14]:
def train():
    for step in range(102400):
        _, train_loss_value = sess.run([train_op, train_loss])

        if (step + 1) % 128 == 0:
            print("train loss:{}".format(train_loss_value))

        if (step + 1) % 128 == 0:
            sess.run([val_loss, train_mean_abs_dist, val_mean_abs_dist])

        if (step + 1) % 128 == 0:
            summary_str = sess.run(summary_op)
            summary_writer.add_summary(summary_str, step)
            summary_writer.flush()

In [15]:
train()

train loss:278.2732849121094
train loss:261.6927490234375
train loss:255.8833465576172
train loss:252.3308563232422
train loss:249.5305633544922
train loss:247.08425903320312
train loss:244.8546905517578
train loss:242.78880310058594
train loss:240.86715698242188
train loss:239.0858154296875
train loss:237.4473114013672
train loss:235.95668029785156
train loss:234.61712646484375
train loss:233.4282684326172
train loss:232.3843231201172
train loss:231.47535705566406
train loss:230.68966674804688
train loss:230.0154266357422
train loss:229.44252014160156
train loss:228.9625244140625
train loss:228.56793212890625


KeyboardInterrupt: 